To create the Restaurant recommendation system, I will create a content-based recommendation system where when I enter the name of a restaurant, the Restaurant recommendation system will look at reviews from other restaurants, and System will recommend us to the other restaurants with similar reviews and sort them from the top-rated.

In [5]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import re

import warnings
warnings.filterwarnings('always')
warnings.filterwarnings('ignore')
from sklearn.metrics.pairwise import linear_kernel
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer

In [6]:
df=pd.read_csv('zomato.csv')

In [7]:
df.head()

,url,address,name,online_order,book_table,rate,votes,phone,location,rest_type,dish_liked,cuisines,approx_cost(for two people),reviews_list,menu_item,listed_in(type),listed_in(city)
0,https://www.zomato.com/bangalore/jalsa-banasha...,"942, 21st Main Road, 2nd Stage, Banashankari, ...",Jalsa,Yes,Yes,4.1/5,775,080 42297555\r\n+91 9743772233,Banashankari,Casual Dining,"Pasta, Lunch Buffet, Masala Papad, Paneer Laja...","North Indian, Mughlai, Chinese",800,"[('Rated 4.0', 'RATED\n A beautiful place to ...",[],Buffet,Banashankari
1,https://www.zomato.com/bangalore/spice-elephan...,"2nd Floor, 80 Feet Road, Near Big Bazaar, 6th ...",Spice Elephant,Yes,No,4.1/5,787,080 41714161,Banashankari,Casual Dining,"Momos, Lunch Buffet, Chocolate Nirvana, Thai G...","Chinese, North Indian, Thai",800,"[('Rated 4.0', 'RATED\n Had been here for din...",[],Buffet,Banashankari
2,https://www.zomato.com/SanchurroBangalore?cont...,"1112, Next to KIMS Medical College, 17th Cross...",San Churro Cafe,Yes,No,3.8/5,918,+91 9663487993,Banashankari,"Cafe, Casual Dining","Churros, Cannelloni, Minestrone Soup, Hot Choc...","Cafe, Mexican, Italian",800,"[('Rated 3.0', ""RATED\n Ambience is not that ...",[],Buffet,Banashankari
3,https://www.zomato.com/bangalore/addhuri-udupi...,"1st Floor, Annakuteera, 3rd Stage, Banashankar...",Addhuri Udupi Bhojana,No,No,3.7/5,88,+91 9620009302,Banashankari,Quick Bites,Masala Dosa,"South Indian, North Indian",300,"[('Rated 4.0', ""RATED\n Great food and proper...",[],Buffet,Banashankari
4,https://www.zomato.com/bangalore/grand-village...,"10, 3rd Floor, Lakshmi Associates, Gandhi Baza...",Grand Village,No,No,3.8/5,166,+91 8026612447\r\n+91 9901210005,Basavanagudi,Casual Dining,"Panipuri, Gol Gappe","North Indian, Rajasthani",600,"[('Rated 4.0', 'RATED\n Very good restaurant ...",[],Buffet,Banashankari


In [8]:
df.shape

(51717, 17)

In [9]:
df.columns

Index(['url', 'address', 'name', 'online_order', 'book_table', 'rate', 'votes',
       'phone', 'location', 'rest_type', 'dish_liked', 'cuisines',
       'approx_cost(for two people)', 'reviews_list', 'menu_item',
       'listed_in(type)', 'listed_in(city)'],
      dtype='object')

In [10]:
zomato=df.drop(['url','dish_liked','phone'],axis=1)

In [11]:
zomato.duplicated().sum()

43

In [12]:
zomato.drop_duplicates(inplace=True)

In [13]:
zomato.duplicated().sum()

0

In [14]:
zomato.isnull().sum()

address                           0
name                              0
online_order                      0
book_table                        0
rate                           7767
votes                             0
location                         21
rest_type                       227
cuisines                         45
approx_cost(for two people)     345
reviews_list                      0
menu_item                         0
listed_in(type)                   0
listed_in(city)                   0
dtype: int64

In [15]:
zomato.dropna(inplace=True)

In [16]:
zomato.isnull().sum()

address                        0
name                           0
online_order                   0
book_table                     0
rate                           0
votes                          0
location                       0
rest_type                      0
cuisines                       0
approx_cost(for two people)    0
reviews_list                   0
menu_item                      0
listed_in(type)                0
listed_in(city)                0
dtype: int64

In [17]:
zomato=zomato.rename(columns={'approx_cost(for two people)':'cost','listed_in(type)':'type', 'listed_in(city)':'city'})

In [18]:
zomato.columns

Index(['address', 'name', 'online_order', 'book_table', 'rate', 'votes',
       'location', 'rest_type', 'cuisines', 'cost', 'reviews_list',
       'menu_item', 'type', 'city'],
      dtype='object')

In [19]:
zomato['cost']=zomato['cost'].astype(str)
zomato['cost']=zomato['cost'].apply(lambda x:x.replace(',','.'))
zomato['cost']=zomato['cost'].astype(float)
zomato['cost']

0        800.0
1        800.0
2        800.0
3        300.0
4        600.0
         ...  
51709    800.0
51711    800.0
51712      1.5
51715      2.5
51716      1.5
Name: cost, Length: 43499, dtype: float64

In [20]:
zomato['rate']

0         4.1/5
1         4.1/5
2         3.8/5
3         3.7/5
4         3.8/5
          ...  
51709    3.7 /5
51711    2.5 /5
51712    3.6 /5
51715    4.3 /5
51716    3.4 /5
Name: rate, Length: 43499, dtype: object

In [21]:
#Removing '/5' from ratings
zomato= zomato.loc[zomato.rate !='NEW']
zomato=zomato.loc[zomato.rate !='-'].reset_index(drop=True)
remove_slash = lambda x:x.replace('/5','') if type(x)==np.str else x
zomato.rate= zomato.rate.apply(remove_slash).str.strip().astype('float')

In [22]:
#djusting the column names
zomato.name=zomato.name.apply(lambda x:x.title())
zomato.online_order.replace(('Yes','No'),('True','False'),inplace=True)
zomato.book_table.replace(('Yes','No'),('True','False'),inplace=True)

In [23]:
zomato.head(5)

,address,name,online_order,book_table,rate,votes,location,rest_type,cuisines,cost,reviews_list,menu_item,type,city
0,"942, 21st Main Road, 2nd Stage, Banashankari, ...",Jalsa,True,True,4.1,775,Banashankari,Casual Dining,"North Indian, Mughlai, Chinese",800.0,"[('Rated 4.0', 'RATED\n A beautiful place to ...",[],Buffet,Banashankari
1,"2nd Floor, 80 Feet Road, Near Big Bazaar, 6th ...",Spice Elephant,True,False,4.1,787,Banashankari,Casual Dining,"Chinese, North Indian, Thai",800.0,"[('Rated 4.0', 'RATED\n Had been here for din...",[],Buffet,Banashankari
2,"1112, Next to KIMS Medical College, 17th Cross...",San Churro Cafe,True,False,3.8,918,Banashankari,"Cafe, Casual Dining","Cafe, Mexican, Italian",800.0,"[('Rated 3.0', ""RATED\n Ambience is not that ...",[],Buffet,Banashankari
3,"1st Floor, Annakuteera, 3rd Stage, Banashankar...",Addhuri Udupi Bhojana,False,False,3.7,88,Banashankari,Quick Bites,"South Indian, North Indian",300.0,"[('Rated 4.0', ""RATED\n Great food and proper...",[],Buffet,Banashankari
4,"10, 3rd Floor, Lakshmi Associates, Gandhi Baza...",Grand Village,False,False,3.8,166,Basavanagudi,Casual Dining,"North Indian, Rajasthani",600.0,"[('Rated 4.0', 'RATED\n Very good restaurant ...",[],Buffet,Banashankari


In [24]:
#Computing Mean rating
restaurants=list(zomato['name'].unique())
zomato['Mean_rating']=0

for i in range(len(restaurants)):
    zomato['Mean_rating'][zomato['name']==restaurants[i]]=zomato['rate'][zomato['name']==restaurants[i]].mean()

In [25]:
from sklearn.preprocessing import MinMaxScaler
scaler=MinMaxScaler(feature_range=(1,5))
zomato[['Mean_rating']]=scaler.fit_transform(zomato[['Mean_rating']]).round(2)

In [26]:
#Lower casing
zomato['reviews_list']=zomato['reviews_list'].str.lower()

In [27]:
import string
PUNCT_TO_REMOVE=string.punctuation
def remove_punctuation(text):
    return text.translate(str.maketrans('','',PUNCT_TO_REMOVE))

zomato['reviews_list']=zomato['reviews_list'].apply(lambda text: remove_punctuation(text))

In [28]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\admin\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [29]:
#Removal of stopwords
from nltk.corpus import stopwords
STOPWORDS= set(stopwords.words('english'))
def remove_stopwords(text):
    return ''.join([word for word in str(text).split() if word not in STOPWORDS])

zomato['reviews_list']= zomato['reviews_list'].apply(lambda text:remove_stopwords(text))

In [30]:
#Removal of Urls
def remove_urls(text):
    url_pattern=re.compile(r'https?://\S+|www\.\S+')
    return url_pattern.sub(r'',text)

zomato['reviews_list']= zomato['reviews_list'].apply(lambda text:remove_urls(text))

In [31]:
zomato[['reviews_list','cuisines']].sample(10)

,reviews_list,cuisines
40107,rated40ratednwentsaturdaylunchorderedvegchicke...,"Biryani, North Indian, Kebab"
15040,rated50ratednonebestindoripohasbangalorendefin...,"Street Food, Fast Food"
6098,rated50ratednfoodtasteyummyqualityquantitywort...,"North Indian, Bengali, Oriya"
26265,rated40ratednplacenarrowstreetindiranagarfishc...,"Oriya, North Indian, Bengali, Chinese"
8872,rated50ratednutterlydeliciousbengalifoodordere...,"Bengali, North Indian"
39280,rated40ratedngoodfoodbudgetorderedhomedelivery...,Lebanese
792,rated20ratedn15tbhgobichillmushroomchilliuptom...,"Chinese, North Indian"
32110,,South Indian
30530,rated50ratedn5starsnamequalityquantityicecream...,"Desserts, Fast Food, Beverages"
9547,rated40ratednhealthyfreshprovidegoodoptionsple...,"Mexican, Fast Food"


In [32]:
##Restaurant Names

restaurant_names = list(zomato['name'].unique())
def get_top_words(column,top_nu_of_words,nu_of_word):
    vec=CountVectorizer(ngram_range=nu_of_word,stop_words='english')
    bag_of_words=vec.fit_transform(column)
    sum_words=bag_of_words.sum(axis=0)
    words_freq=[(word,sum_words[0,idx]) for word,idx in vec.vocabulary_.items()]
    words_freq=sorted(words_freq,key=lambda x:x[1],reverse=True)
    return words_freq[:top_nu_of_words]

In [33]:
zomato=zomato.drop(['address','rest_type', 'type', 'menu_item', 'votes'],axis=1)

In [34]:
df_percent= zomato.sample(frac=0.5)

In [35]:
df_percent.set_index('name',inplace=True)
df_percent

,online_order,book_table,rate,location,cuisines,cost,reviews_list,city,Mean_rating
name,,,,,,,,,
55 Wall Street,False,True,3.9,JP Nagar,"Finger Food, North Indian, Chinese",1.3,rated50ratedntwowaysenjoyonefloordiscenjoylive...,BTM,3.71
Chili'S American Grill & Bar,True,True,4.6,Bellandur,"American, Tex-Mex, Burger, BBQ",1.8,rated40ratedngoodtimesspandherenwecomefriendsf...,Sarjapur Road,4.70
The Terminus - The Oterra,False,True,4.0,Electronic City,"Finger Food, Continental",2.5,rated50ratednexcellentplacebitfarmaincityuvisi...,Electronic City,3.84
Shanthi Sagar,True,False,4.0,Ulsoor,"South Indian, North Indian, Chinese, Juices",250.0,rated10ratednpaneermasalapaneerportantthingadd...,Residency Road,3.44
Hotel Salala,True,False,3.6,Brookefield,"North Indian, South Indian, Chinese, Biryani",400.0,rated50ratedngoodrestaurantlovekeraliancuisine...,Marathahalli,3.39
...,...,...,...,...,...,...,...,...,...
Hotel Devanand Kerala Restaurant,True,False,3.4,CV Raman Nagar,Kerala,300.0,rated10ratednoneworstkeralafoodorderedswiggypu...,Old Airport Road,3.06
CafãÂÃÂÃÂÃÂÃÂÃÂÃÂÃÂÃÂÃÂÃÂÃÂÃÂÃÂÃÂÃÂ© Mor'Ish,True,False,4.1,Richmond Road,"BBQ, Continental, Burger, Wraps, Pizza, Cafe",700.0,rated40ratednofficefrndsorderedminifalafelpita...,Lavelle Road,3.97
Foxtrot - House Of Subculture,False,True,4.4,Koramangala 3rd Block,"Cafe, American, Asian, North Indian",1.0,rated50ratednambience55nlocation55nfood55nserv...,Koramangala 5th Block,4.35


In [36]:
indices=pd.Series(df_percent.index)
indices

0                                           55 Wall Street
1                             Chili'S American Grill & Bar
2                                The Terminus - The Oterra
3                                            Shanthi Sagar
4                                             Hotel Salala
                               ...                        
20613                     Hotel Devanand Kerala Restaurant
20614    CafãÂÃÂÃÂÃÂÃÂÃÂÃÂÃÂÃÂÃÂÃÂ...
20615                        Foxtrot - House Of Subculture
20616    CafãÂÃÂÃÂÃÂÃÂÃÂÃÂÃÂÃÂÃÂÃÂ...
20617                           I-Bar - The Park Bangalore
Name: name, Length: 20618, dtype: object

In [ ]:
from sklearn.metrics.pairwise import linear_kernel
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer

#Creating Tf-IDF matrix
tfidf = TfidfVectorizer(analyzer='word', ngram_range=(1, 2), min_df=0, stop_words='english')
tfidf_matrix=tfidf.fit_transform(df_percent['reviews_list'])

cosine_similarities=linear_kernel(tfidf_matrix,tfidf_matrix)

#Now the last step for creating a Restaurant Recommendation System is to write a function that will recommend restaurants:

In [39]:
def recommend(name, cosine_similarities = cosine_similarities):
    
    # Create a list to put top restaurants
    recommend_restaurant = []
    
    # Find the index of the hotel entered
    idx = indices[indices == name].index[0]
    
    # Find the restaurants with a similar cosine-sim value and order them from bigges number
    score_series = pd.Series(cosine_similarities[idx]).sort_values(ascending=False)
    
    # Extract top 30 restaurant indexes with a similar cosine-sim value
    top30_indexes = list(score_series.iloc[0:31].index)
    
    # Names of the top 30 restaurants
    for each in top30_indexes:
        recommend_restaurant.append(list(df_percent.index)[each])
    
    # Creating the new data set to show similar restaurants
    df_new = pd.DataFrame(columns=['cuisines', 'Mean Rating', 'cost'])
    
    # Create the top 30 similar restaurants with some of their columns
    for each in recommend_restaurant:
        df_new = df_new.append(pd.DataFrame(df_percent[['cuisines','Mean Rating', 'cost']][df_percent.index == each].sample()))
    
    # Drop the same named restaurants and sort only the top 10 by the highest rating
    df_new = df_new.drop_duplicates(subset=['cuisines','Mean Rating', 'cost'], keep=False)
    df_new = df_new.sort_values(by='Mean Rating', ascending=False).head(10)
    
    print('TOP %s RESTAURANTS LIKE %s WITH SIMILAR REVIEWS: ' % (str(len(df_new)), name))
    
    return df_new
#recommend('Pai Vihar')